### Train model


In [2]:
import torch
from torch.utils.data import DataLoader 
import numpy as np
import matplotlib.pyplot as plt
import os
from torchmetrics import FBetaScore, Accuracy
import torchvision.transforms as T
from torchvision.transforms import functional as F

import src.dfmaker as dfmaker
import src.BWDatasets as BWDatasets
import src.utility as utility

from paddleocr import PaddleOCR
import paddle

In [3]:
seed = 115
generator = torch.Generator()
generator.manual_seed(seed)
np_generator = np.random.default_rng(seed=seed)

In [4]:
device = torch.device('cpu')

# Make sure the notebook is deterministic if training on gpu
if torch.cuda.is_available():
    device = torch.device('cuda')
    os.environ["CUBLAS_WORKSPACE_CONFIG"] = ":4096:8"
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False

print(f'Cuda availability is: {torch.cuda.is_available()}')  # Returns True if a GPU is available
print(f"Training on device {device}.")

Cuda availability is: True
Training on device cuda.


In [ ]:
# Lyngøy
label_path_lyng = "dataset/datasets/lyngoy/ringcodes.csv"
image_path_lyng = "dataset/datasets/lyngoy/images"
bb_path_lyng = "dataset/datasets/lyngoy/labels"

# RF
label_path_rf = "dataset/datasets/rf/ringcodes.csv"
image_path_rf = "dataset/datasets/rf/images"
bb_path_rf = "dataset/datasets/rf/labels"

# Ringmerkingno
label_path_rno = "dataset/datasets/ringmerkingno/ringcodes.csv"
image_path_rno = "dataset/datasets/ringmerkingno/images"
bb_path_rno = "dataset/datasets/ringmerkingno/labels"

max_n_single = 10
max_n_all = None
height = 240 # 240 worked best for the provided dataset

df_lyng = dfmaker.make_dataframe(labels_path=label_path_lyng, img_path=image_path_lyng, bb_path=bb_path_lyng, max_n=max_n_single)
df_rf = dfmaker.make_dataframe(labels_path=label_path_rf, img_path=image_path_rf, bb_path=bb_path_rf, max_n=max_n_single)
df_rno = dfmaker.make_dataframe(labels_path=label_path_rno, img_path=image_path_rno, bb_path=bb_path_rno, max_n=max_n_single)
complete_df = dfmaker.combine_dfs([df_lyng, df_rf, df_rno])

# Shuffle the dataset to remove order of elements
df_shuffled = complete_df.sample(frac=1).reset_index(drop=True)

# Calculate the splits
train_factor = 0.7
val_factor = 0.15
test_factor = 0.15

train_n = int(train_factor*len(df_shuffled))
val_n = int(val_factor*len(df_shuffled))
test_n = int(test_factor*len(df_shuffled))

# Split data into three dataframes
df_train = df_shuffled.iloc[:train_n]
df_val = df_shuffled.iloc[train_n:train_n+val_n]
df_test = df_shuffled.iloc[train_n+val_n:]

In [15]:
print(df_train)
print(df_val)
print(df_test)
print(df_shuffled)


                                           filename  \
0         20240408-200419(127.41,-12.78,4760.0).jpg   
1          20240409-054649(134.06,-6.44,8406.0).jpg   
2        20180709_gramake_3k_JL435_lyngoy_ost2_.JPG   
3         20240408-195001(120.36,-15.82,3091.0).jpg   
4       20180702_gramake_4k+_JL177_lyngoy_ost4_.JPG   
..                                              ...   
205  20180702_svartbak_4k+_JJ165_lyngoy_vest14_.JPG   
206   20180705_sildemake_4k+_J236H_lyngoy_midt_.JPG   
207                LBBW(J1X4)_20090311224947_79.jpg   
208     20180703_gramake_4k+_JHE35_lyngoy_midt_.JPG   
209   20180702_svartbak_4k+_JJ165_lyngoy_vest2_.JPG   

                                             img_paths  \
0    dataset/datasets/rf/images\20240408-200419(127...   
1    dataset/datasets/rf/images\20240409-054649(134...   
2    dataset/datasets/lyngoy/images\20180709_gramak...   
3    dataset/datasets/rf/images\20240408-195001(120...   
4    dataset/datasets/lyngoy/images\20180702_gram

In [14]:
transform = T.Compose([
    T.Lambda(lambda x: F.rotate(x, -90, expand=True)),
    T.Resize((height, utility.find_width(height))) 
])

train_ds = BWDatasets.TrainDataSet(df=df_train, transform=transform, max_n=max_n_all)
val_ds = BWDatasets.TrainDataSet(df=df_val, transform=transform, max_n=max_n_all)
test_ds = BWDatasets.TrainDataSet(df=df_test, transform=transform, max_n=max_n_all)
